In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim, nn
import wandb
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

In [2]:
wandb.login()

wandb: Currently logged in as: moritz-palm. Use `wandb login --relogin` to force relogin


True

In [3]:
config = {
    "learning_rate": 0.01,
    "architecture": "NN",
    "dataset": "static_1.1",
    "epochs": 500,
    "classes": 2,
    "batch_size": 64,
    "num_layers": 2,
    "hidden_size": 128,
    "dropout_prob": 0.2,
    "input_size": 99,
    "output_size": 1,
    "optimizer": "Adam",
    "loss": "CrossEntropyLoss",
    "activation": "ReLU",
}

In [4]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.transform = transform
        self.target_transform = target_transform
        print(f'shape of data: {self.data.shape}')
        print(f'shape of labels: {self.labels.shape}')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label


In [5]:
def get_train_data(val_split=0.8) -> (Dataset, Dataset):
    """
    Returns a train and validation dataset
    :param val_split: percentage of data to use for validation
    :return: train_dataset, val_dataset
    """
    dataset = StaticDataset('../data/processed/train_static.npy')
    train_len = int(len(dataset) * val_split)
    val_len = len(dataset) - train_len
    print(f'train_len: {train_len}, val_len: {val_len}')
    return torch.utils.data.random_split(dataset, [train_len, val_len])

In [6]:
def get_test_data() -> Dataset:
    """
    Returns a test dataset
    :return: test_dataset
    """
    return StaticDataset('../data/processed/test_static.npy')

In [7]:
def make_loader(dataset, batch_size=64) -> DataLoader:
    """
    Returns a DataLoader for the given dataset
    :param dataset:
    :param batch_size:
    :return:
    """
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, num_classes=2):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential()
        self.linear_relu_stack.append(nn.Linear(input_size, hidden_size))
        self.linear_relu_stack.append(nn.ReLU())
        self.linear_relu_stack.append(nn.Linear(hidden_size, hidden_size))
        self.linear_relu_stack.append(nn.ReLU())
        self.linear_relu_stack.append(nn.Linear(hidden_size, num_classes))
        self.linear_relu_stack.append(nn.Sigmoid())

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [9]:
def train(model, train_loader, val_loader, test_loader, criterion, optimizer, config):
    """
    Trains the given model
    :param model:
    :param train_loader:
    :param val_loader:
    :param criterion:
    :param optimizer:
    :param config:
    :return:
    """
    wandb.watch(model, criterion, log="all", log_freq=10)
    wandb_metrics = {}
    loss_vals = []
    vloss_vals = []
    acc_vals = []
    vacc_vals = []
    since = time.time()
    for epoch in tqdm(range(config.epochs)):
        for phase in ['train', 'val', 'test']:
            if phase == 'train':
                model.train()
                loader = train_loader
            else:
                model.eval()
                loader = val_loader

            running_loss = 0.
            running_corrects = 0

            for i, (matches, labels) in enumerate(loader):
                print(f'matches: {matches.shape}, labels: {labels.shape}')
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(matches)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * matches.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = running_corrects.double() / len(loader.dataset)
            if phase == 'train':
                loss_vals.append(epoch_loss)
                acc_vals.append(epoch_acc)
                wandb_metrics['train_loss'] = epoch_loss
                wandb_metrics['train_acc'] = epoch_acc
            elif phase == 'val':
                vloss_vals.append(epoch_loss)
                vacc_vals.append(epoch_acc)
                wandb_metrics['val_loss'] = epoch_loss
                wandb_metrics['val_acc'] = epoch_acc
            else:
                test(model, test_loader)

    wandb.log(wandb_metrics)
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    plt.plot(loss_vals, label='train')
    plt.plot(vloss_vals, label='val')
    plt.legend()

In [10]:
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct, total = 0, 0
        y_pred = []
        y_true = []
        for matches, labels in test_loader:
            matches, labels = matches.to(device), labels.to(device)
            outputs = model(matches)
            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(classification_report(y_pred, y_true))

        print(f"Accuracy of the model on the {total} " +
              f"test matches: {correct / total:%}")

        wandb.log({"test_accuracy": correct / total})

In [11]:
def make(config) -> (NeuralNetwork, DataLoader, DataLoader, nn.CrossEntropyLoss, optim.Optimizer):
    """
    Returns a model, train_loader, val_loader, criterion, optimizer
    :param config:
    :return:
    """
    train_dataset, val_dataset = get_train_data()
    train_loader = make_loader(train_dataset, config.batch_size)
    val_loader = make_loader(val_dataset, config.batch_size)
    test_dataset = get_test_data()
    test_loader = make_loader(test_dataset, config.batch_size)
    model = NeuralNetwork(config.input_size, config.hidden_size, config.num_layers, config.dropout_prob,
                          config.output_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    return model, train_loader, val_loader, test_loader, criterion, optimizer

In [12]:
def model_pipeline(hyperparameters):
    with wandb.init(project='leaguify', config=hyperparameters):
        config = wandb.config
        global device
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"Using {device} for training")

        model, train_loader, val_loader, test_loader, criterion, optimizer = make(config)
        print(model)

        train(model, train_loader, val_loader, test_loader, criterion, optimizer, config)

In [13]:
model = model_pipeline(config)

Using cuda:0 for training
shape of data: torch.Size([15960, 100])
shape of labels: torch.Size([15960])
train_len: 12768, val_len: 3192
shape of data: torch.Size([3990, 100])
shape of labels: torch.Size([3990])
NeuralNetwork(
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=99, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


  0%|          | 0/500 [00:00<?, ?it/s]

matches: torch.Size([64, 99]), labels: torch.Size([64])


  0%|          | 0/500 [00:01<?, ?it/s]
Traceback (most recent call last):
  File "C:\Users\morit\AppData\Local\Temp\ipykernel_28880\2898729893.py", line 11, in model_pipeline
    train(model, train_loader, val_loader, test_loader, criterion, optimizer, config)
  File "C:\Users\morit\AppData\Local\Temp\ipykernel_28880\2512386789.py", line 39, in train
    loss.backward()
  File "C:\Users\morit\AppData\Local\pypoetry\Cache\virtualenvs\leaguify-VaCbhr8h-py3.11\Lib\site-packages\torch\_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "C:\Users\morit\AppData\Local\pypoetry\Cache\virtualenvs\leaguify-VaCbhr8h-py3.11\Lib\site-packages\torch\autograd\__init__.py", line 251, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`